In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as de
import os

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import seaborn as sns
sns.set(context='paper', style='darkgrid', rc={'figure.facecolor':'white'}, font_scale=1.2)

In [12]:
win_path = 'C:/Users/noname/Desktop/wuzhatn'
mac_path = '/Users/noname/documents/github/wuzhatn'
main = win_path if os.path.exists(win_path) else mac_path
raw = main + '/raw'

print(raw)

years = range(2011,2019,1)
y2u = range(2014,2019,1)

/Users/noname/documents/github/wuzhatn/raw


In [79]:
all_ds = {}
for y in y2u:
    all_ds[y] = pd.read_excel(f'{raw}/ExcelData_{str(y)}.xlsx', skiprows=3, header=1, usecols=[0,1,2,3])
    all_ds[y].columns = ['dt','loca','pm10','pm25']
    all_ds[y] = all_ds[y][all_ds[y]['dt']!='전체']
    
ds = pd.DataFrame()
for y in y2u:
    ds = ds.append(all_ds[y])

ds.columns = ['dt','loca','pm10','pm25']
ds['dt'] = pd.to_datetime(ds['dt'], format='%Y%m%d')

ds = ds[ds['loca'].str[-1]=='구']
ds.reset_index(inplace=True, drop=True)

st = pd.to_datetime(f'{min(y2u)}0101',format='%Y%m%d')
ed = pd.to_datetime(f'{max(y2u)}1231',format='%Y%m%d')
dates = []
i = st
while i <= ed:
    dates.append(i)
    i += de.timedelta(days=1)
    
denom = pd.DataFrame( [[x,y] for x in dates for y in locas], columns=['dt','loca'])
ds = denom.merge(ds, on=['dt','loca'], how='outer', validate='1:1')

ds['day'] = (ds['dt'] - pd.to_datetime(ds['dt'].dt.year.astype(str)+'0101', format='%Y%m%d')).dt.days
ds['y'] = ds['dt'].dt.year
ds['m'] = ds['dt'].dt.month

locas = ['서울전체'] + list(ds.loca.unique())
ds.head()

,dt,loca,pm10,pm25
1,2014-12-31 00:00:00,평균,37.0,16.0
2,2014-12-31 00:00:00,종로구,40.0,14.0
3,2014-12-31 00:00:00,중구,34.0,12.0
4,2014-12-31 00:00:00,용산구,35.0,15.0
5,2014-12-31 00:00:00,성동구,38.0,18.0


In [100]:
all_ds = {}
for y in y2u:
    all_ds[y] = pd.read_excel(f'{raw}/ExcelData_{str(y)}.xlsx', skiprows=3, header=1, usecols=[0,1,2,3])
    all_ds[y].columns = ['dt','loca','pm10','pm25']
    all_ds[y] = all_ds[y][all_ds[y]['dt']!='전체']

ori_ds = {}
for y in years:
    ori_ds[y] = pd.read_excel(f'{raw}/일별평균대기오염도_{str(y)}.xlsx', usecols=[0,1,6,7])
    ori_ds[y].dropna(axis=1, how='all', inplace=True)
    
def append_ds(all_ds, y2u):
    ds = pd.DataFrame()
    for y in y2u:
        ds = ds.append(all_ds[y])

    ds.columns = ['dt','loca','pm10','pm25']
    ds['dt'] = pd.to_datetime(ds['dt'], format='%Y%m%d')

    ds = ds[ds['loca'].str[-1]=='구']
    ds.reset_index(inplace=True, drop=True)
    
    locas = ds.loca.unique()

    st = pd.to_datetime(f'{min(y2u)}0101',format='%Y%m%d')
    ed = pd.to_datetime(f'{max(y2u)}1231',format='%Y%m%d')
    dates = []
    i = st
    while i <= ed:
        dates.append(i)
        i += de.timedelta(days=1)

    denom = pd.DataFrame( [[x,y] for x in dates for y in locas], columns=['dt','loca'])
    ds = denom.merge(ds, on=['dt','loca'], how='outer', validate='1:1')

    ds['day'] = (ds['dt'] - pd.to_datetime(ds['dt'].dt.year.astype(str)+'0101', format='%Y%m%d')).dt.days
    ds['y'] = ds['dt'].dt.year
    ds['m'] = ds['dt'].dt.month

    locas = ['서울전체'] + list(ds.loca.unique())
    return ds

In [118]:
ori = append_ds(ori_ds, y2u)
new = append_ds(all_ds, y2u)

og = ori[(ori['loca']=='강남구') & (ori['dt'].dt.year==2018)]
yb = new[(new['loca']=='강남구') & (new['dt'].dt.year==2018)]
md = og[['dt','pm10','pm25']].merge(yb[['dt','pm10','pm25']], on=['dt'],suffixes=('_og','_yb'),validate='1:1')
for v in ['pm10','pm25']:
    md[v+'_diff'] = (md[v+'_og'] - md[v+'_yb']).abs()

print(md[(md['pm10_diff']>0) | (md['pm25_diff']>0)].shape)
print(md[(md['pm10_diff']==1) | (md['pm25_diff']==1)].shape)
print(md[(md['pm10_diff']>1) | (md['pm25_diff']>1)].shape)
print(md.pm10_diff.describe())
print(md.pm25_diff.describe())
md.sort_values('pm10_diff',ascending=False).head(10)
md[(md['pm10_diff']>1) | (md['pm25_diff']>1)]

(154, 7)
(133, 7)
(30, 7)
count    309.000000
mean     0.624595  
std      1.162807  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      1.000000  
max      9.000000  
Name: pm10_diff, dtype: float64
count    309.000000
mean     0.427184  
std      0.973032  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      1.000000  
max      7.000000  
Name: pm25_diff, dtype: float64


,dt,pm10_og,pm25_og,pm10_yb,pm25_yb,pm10_diff,pm25_diff
8,2018-01-09,36.0,20.0,34.0,19.0,2.0,1.0
21,2018-01-22,38.0,28.0,42.0,32.0,4.0,4.0
46,2018-02-16,43.0,17.0,41.0,22.0,2.0,5.0
58,2018-02-28,63.0,48.0,61.0,46.0,2.0,2.0
66,2018-03-08,29.0,24.0,30.0,26.0,1.0,2.0
104,2018-04-15,69.0,19.0,67.0,19.0,2.0,0.0
107,2018-04-18,63.0,34.0,65.0,34.0,2.0,0.0
108,2018-04-19,81.0,37.0,77.0,36.0,4.0,1.0
110,2018-04-21,70.0,41.0,68.0,40.0,2.0,1.0
121,2018-05-02,43.0,37.0,41.0,35.0,2.0,2.0


In [82]:
locas = ds.loca.unique()

st = pd.to_datetime(f'{min(y2u)}0101',format='%Y%m%d')
ed = pd.to_datetime(f'{max(y2u)}1231',format='%Y%m%d')
dates = []
i = st
while i <= ed:
    dates.append(i)
    i += de.timedelta(days=1)
print(dates[-1])
print(len(dates))
print(locas)

denom = pd.DataFrame( [[x,y] for x in dates for y in locas], columns=['dt','loca'])
print(denom.shape)
print(type(denom.dt[0]))
print(ds.shape)
ds = denom.merge(ds, on=['dt','loca'], how='outer', validate='1:1')
print(ds.shape)

ds['day'] = (ds['dt'] - pd.to_datetime(ds['dt'].dt.year.astype(str)+'0101', format='%Y%m%d')).dt.days
ds['y'] = ds['dt'].dt.year
ds['m'] = ds['dt'].dt.month

locas = ['서울전체'] + list(ds.loca.unique())
ds.head()

2018-12-31 00:00:00
1826
['종로구' '중구' '용산구' '성동구' '광진구' '동대문구' '중랑구' '성북구' '강북구' '도봉구' '노원구' '은평구'
 '서대문구' '마포구' '양천구' '강서구' '구로구' '금천구' '영등포구' '동작구' '관악구' '서초구' '강남구' '송파구'
 '강동구']
(45650, 2)
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
(45557, 4)
(45650, 4)


,dt,loca,pm10,pm25,day,y,m
0,2014-01-01,종로구,112.0,21.0,0,2014,1
1,2014-01-01,중구,117.0,27.0,0,2014,1
2,2014-01-01,용산구,119.0,26.0,0,2014,1
3,2014-01-01,성동구,116.0,22.0,0,2014,1
4,2014-01-01,광진구,132.0,27.0,0,2014,1


In [85]:
ds.isnull().sum()

dt      0  
loca    0  
pm10    471
pm25    356
day     0  
y       0  
m       0  
dtype: int64

In [95]:
print(ds.loc[(ds['pm10'].isna()==True) & (ds['dt'].dt.year==2018),'loca'].unique())
ds[ds['pm10'].isna()==True]
ds[(ds['pm10'].isna()==True) & (ds['dt'].dt.year==2018)]
ds[(ds['dt'].dt.year==2018) & (ds['loca']=='강남구')].isnull().sum()

['서대문구' '강서구' '구로구' '서초구' '용산구' '광진구' '성동구' '마포구' '송파구' '종로구']


dt      0
loca    0
pm10    0
pm25    0
day     0
y       0
m       0
dtype: int64

In [12]:
seoul_daily = ds[['dt','pm10','pm25']].groupby(['dt']).mean().reset_index()
seoul_daily['loca'] = '서울전체'
seoul_daily.head()
daily_ds = seoul_daily.append(ds[['dt','loca','pm10','pm25']])
daily_ds['day_value'] = (daily_ds['dt'] - st) / np.timedelta64(1,'D')

daily_ds.head()
daily_dict = {}
for r in locas:
    daily_dict[r] = daily_ds[daily_ds['loca']==r]

daily_ds.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,dt,loca,pm10,pm25,day_value
0,2014-01-01,서울전체,116.68,25.44,0.0
1,2014-01-02,서울전체,51.84,19.32,1.0
2,2014-01-03,서울전체,59.16,25.76,2.0
3,2014-01-04,서울전체,44.48,20.44,3.0
4,2014-01-05,서울전체,42.32,22.28,4.0


In [21]:
for k,v in daily_dict.items():
    print('')
    print(k)
    print(v[v['pm25'].isna()==True].shape)
    
daily_dict['강남구'][daily_dict['강남구']['pm25'].isna()==True]


서울전체
(56, 5)

강남구
(57, 5)

강동구
(57, 5)

강북구
(83, 5)

강서구
(72, 5)

관악구
(58, 5)

광진구
(58, 5)

구로구
(81, 5)

금천구
(85, 5)

노원구
(68, 5)

도봉구
(56, 5)

동대문구
(57, 5)

동작구
(75, 5)

마포구
(63, 5)

서대문구
(100, 5)

서초구
(68, 5)

성동구
(58, 5)

성북구
(74, 5)

송파구
(70, 5)

양천구
(109, 5)

영등포구
(64, 5)

용산구
(71, 5)

은평구
(59, 5)

종로구
(73, 5)

중구
(75, 5)

중랑구
(59, 5)


,dt,loca,pm10,pm25,day_value
21725,2016-05-19,강남구,NaN,NaN,869.0
37700,2018-02-17,강남구,NaN,NaN,1508.0
38500,2018-03-21,강남구,NaN,NaN,1540.0
38525,2018-03-22,강남구,NaN,NaN,1541.0
38550,2018-03-23,강남구,NaN,NaN,1542.0
38575,2018-03-24,강남구,NaN,NaN,1543.0
38600,2018-03-25,강남구,NaN,NaN,1544.0
38625,2018-03-26,강남구,NaN,NaN,1545.0
38650,2018-03-27,강남구,NaN,NaN,1546.0
38675,2018-03-28,강남구,NaN,NaN,1547.0


In [32]:
daily_ds[(daily_ds['day_value']>=1) & (daily_ds['day_value']<=3)]

,dt,loca,pm10,pm25,day_value
1,2014-01-02,서울전체,51.84,19.32,1.0
2,2014-01-03,서울전체,59.16,25.76,2.0
3,2014-01-04,서울전체,44.48,20.44,3.0
25,2014-01-02,강남구,54.00,23.00,1.0
26,2014-01-02,강동구,56.00,21.00,1.0
27,2014-01-02,강북구,48.00,16.00,1.0
28,2014-01-02,강서구,55.00,17.00,1.0
29,2014-01-02,관악구,56.00,23.00,1.0
30,2014-01-02,광진구,57.00,22.00,1.0
31,2014-01-02,구로구,56.00,20.00,1.0


In [10]:
from math import ceil
from math import floor

In [17]:
print(type(i.day))

<class 'int'>


In [22]:
    n_days = (ds.dt.max() - ds.dt.min()).days

    i = st
    c = 0
    day_slider_lab = {}
    while i <= ed:
        if (i.day == 1) & (i.month == 1):
            print('')
            print(i.day)
            print(i.month == 1)
            day_slider_lab[c] = f'{i.year}/{i.month}/{i.day}'

        i += de.timedelta(days=1)
        c += 1

print(day_slider_lab)



1
True

1
True

1
True

1
True

1
True
{0: '2014/1/1', 365: '2015/1/1', 730: '2016/1/1', 1096: '2017/1/1', 1461: '2018/1/1'}


In [ ]:
print(st.month)
print(st.day)
print(st.year)


In [ ]:
for k,v in daily_dict.items():
    print(k)
    break
print(daily_dict['강남구']['dt'])

In [ ]:
print(ds[(ds['dt'].dt.year==2018) & (ds['loca']=='강동구') & (ds['pm10'].isna()==True)].shape )
ds[(ds['dt'].dt.year==2018) & (ds['loca']=='강동구') & (ds['pm10'].isna()==True)].head()
ds.head()

In [ ]:
# collapse at month-district level
mgb = ds[['loca','pm10','pm25','y','m']].groupby(['loca','y','m']).agg(['mean','min','max','median'])
mgb.columns = pd.Index(i[0]+'_'+i[1] for i in mgb.columns.tolist())
mgb.reset_index(inplace=True)

# now get a month-level one based on all districts
agb = ds[['pm10','pm25','y','m']].groupby(['y','m']).agg(['mean','min','max','median'])
agb.columns= pd.Index(i[0]+'_'+i[1] for i in agb.columns.tolist())
agb.reset_index(inplace=True)
agb['loca'] = 'all'

gb = agb.append(mgb).reset_index(drop=True)

gb.head()

In [ ]:
meas_dict = {}
for reg in gb.loca.unique():
    meas_dict[reg] = {}
    for v in ['pm10','pm25']:
        meas_dict[reg][v] = {}
        for meas in ['mean','median','min','max']:
            meas_dict[reg][v][meas] = {}
            for m in range(1,13):
                t = [] 
                for y in y2u:
                    t.append(float(gb.loc[(gb['loca']==reg) & (gb['m']==m) \
                                          & (gb['y']==y), v+'_'+meas]))
                meas_dict[reg][v][meas][m] = t
# print(meas_dict)

In [ ]:
for k,v in meas_dict['all']['pm10']['mean'].items():
    print(k)
print(meas_dict['all']['pm10']['mean'][1])

In [ ]:
ys = [2014,2017]
meas_dict = {}
for reg in gb.loca.unique():
    meas_dict[reg] = {}
    for v in ['pm10','pm25']:
        meas_dict[reg][v] = {}
        for meas in ['mean','median','min','max']:
            meas_dict[reg][v][meas] = {}
            for m in range(1,13):
                t = [] 
                for y in ys:
                    t.append(float(gb.loc[(gb['loca']==reg) & (gb['m']==m) \
                                          & (gb['y']==y), v+'_'+meas]))
                meas_dict[reg][v][meas][m] = t

In [ ]:
a = [3,2,4,5,1,8]
a.sort()
print(a)

In [ ]:
import plotly
plotly.__version__
plotly.tools.set_credentials_file(username='transmutate', api_key='84x96fZlmp68dwfftTHY')

In [ ]:
def plot_monthly_trend_over_the_years(measure,reg,pm):
    from plotly import tools
    import plotly.plotly as py
    import plotly.graph_objs as go

    all_dict = {}
    trace_dict = {}
    for i in range(1,13):
        dstring = '1999' + str(i) + '01' if len(str(i))==2 else '19990' + str(i) + '01'
        dval = pd.to_datetime(dstring, format='%Y%m%d')
        mo_lab = dval.strftime("%b")
        
        if i == 1:
            name_fill_base = '서울전체'
            name_fill_comp = reg
            show_leg_stat = True
        else:
            name_fill_base = ''
            name_fill_comp = ''
            show_leg_stat = False
    
        all_dict[i] = go.Scatter(x = list(y2u),
                                 y = meas_dict['all'][pm][measure][i],
                                 hoverinfo = 'y',
                                 marker = dict(color = 'red'),
                                 name = name_fill_base,
                                 showlegend = show_leg_stat)
    
        trace_dict[i] = go.Scatter(x = list(y2u),
                                   y = meas_dict[reg][pm][measure][i],
                                   hoverinfo = 'y',
                                   marker = dict(color = 'blue'),
                                   name = name_fill_comp,
                                   showlegend = show_leg_stat)

    fig = tools.make_subplots(rows=1, cols=12, shared_yaxes=True, \
                              subplot_titles=('Jan','Feb','Mar','Apr','May','Jun',
                                              'Jul','Aug','Sep','Oct','Nov','Dec'))
    for i in range(1,13):
        fig.append_trace(all_dict[i], 1, i)
        fig.append_trace(trace_dict[i], 1, i)
    
    meas_txt_dict = {'min':'Minimum','max':'Maximum','mean':'Average','median':'Median'}
    pm_txt_dict = {'pm10':'PM10','pm25':'PM2.5'}
    title_txt = f'{meas_txt_dict[measure]} {pm_txt_dict[pm]} Level: Seoul vs {reg}'

    fig['layout'].update(height=400, width=1200, title=title_txt)
    fig['layout']['yaxis1'].update(title=f'{pm_txt_dict[pm]} Level in {unit}')

    return(py.iplot(fig, filename='multiple-subplots-shared-yaxes'))

    
    
print('****************************')
print('****************************')
print('****************************')
plot_monthly_trend_over_the_years('median','동작구','pm25')

In [ ]:
measure = 'min'
reg = '송파구'
pm = 'pm25'


from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

all_dict = {}
trace_dict = {}
rc = 0
for reg in gb.loca.unique():
    trace_dict[reg] = {}
    for i in range(1,13):
        dstring = '1999' + str(i) + '01' if len(str(i))==2 else '19990' + str(i) + '01'
        dval = pd.to_datetime(dstring, format='%Y%m%d')
        mo_lab = dval.strftime("%b")

        if i == 1:
            name_fill = reg
            show_leg_stat = True
        else:
            name_fill = ''
            show_leg_stat = False

        trace_dict[reg][i] = go.Scatter(x = list(y2u),
                                        y = meas_dict[reg][pm][measure][i],
                                        hoverinfo = 'y',
                                        marker = dict(color = c2u[rc]),
                                        name = name_fill,
                                        showlegend = show_leg_stat)
    rc += 1

fig = tools.make_subplots(rows=1, cols=12, shared_yaxes=True, \
                          subplot_titles=('Jan','Feb','Mar','Apr','May','Jun',
                                          'Jul','Aug','Sep','Oct','Nov','Dec'))
for reg in gb.loca.unique():
    for i in range(1,13):
        fig.append_trace(trace_dict[reg][i],1,i)

fig['layout'].update(height=400, width=1200, title='Multiple Subplots with Shared Y-Axes')
fig['layout']['yaxis1'].update(title='PM10 Level in whatever da unit')

py.iplot(fig, filename='multiple-subplots-shared-yaxes')


In [ ]:
for k,v in fig.items():
    print(k)

In [ ]:
measure = 'max'
reg = '송파구'
pm = 'pm25'

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

all_dict = {}
trace_dict = {}
for i in range(1,13):
    dstring = '1999' + str(i) + '01' if len(str(i))==2 else '19990' + str(i) + '01'
    dval = pd.to_datetime(dstring, format='%Y%m%d')
    mo_lab = dval.strftime("%b")
    
    if i == 1:
        name_fill_base = '서울전체'
        name_fill_comp = reg
        show_leg_stat = True
    else:
        name_fill_base = ''
        name_fill_comp = ''
        show_leg_stat = False
    
    all_dict[i] = go.Scatter(x = list(y2u),
                             y = meas_dict['all'][pm][measure][i],
                             hoverinfo = 'y',
                             marker = dict(color = 'red'),
                             name = name_fill_base,
                             showlegend = show_leg_stat)
    
    trace_dict[i] = go.Scatter(x = list(y2u),
                               y = meas_dict[reg][pm][measure][i],
                               hoverinfo = 'y',
                               marker = dict(color = 'blue'),
                               name = name_fill_comp,
                               showlegend = show_leg_stat)

fig = tools.make_subplots(rows=1, cols=12, shared_yaxes=True, \
                          subplot_titles=('Jan','Feb','Mar','Apr','May','Jun',
                                          'Jul','Aug','Sep','Oct','Nov','Dec'))
for i in range(1,13):
    fig.append_trace(all_dict[i], 1, i)
    fig.append_trace(trace_dict[i], 1, i)

fig['layout'].update(height=400, width=1200, title='Multiple Subplots with Shared Y-Axes')
fig['layout']['yaxis1'].update(title='PM10 Level in whatever da unit')

py.iplot(fig, filename='multiple-subplots-shared-yaxes')


In [ ]:
print(fig)

In [ ]:
plot_monthly_trend_over_the_years('mean','강북구','pm10')

In [ ]:
plot_monthly_trend_over_the_years('mean','중랑구','pm10')

In [ ]:
print(gb.loca.unique())

In [ ]:
print(y2u)
plt.subplots(figsize=(16,5))
# plt.xticks(ds.loc[(ds['dt'].dt.year==2015) & (ds['loca']=='강남구'),'dt'].tolist(), rotation=90)
for yee in y2u:
    plt.plot(mgb.loc[(mgb['y']==yee) & (mgb['loca']=='강남구'),'m'],\
             mgb.loc[(mgb['y']==yee) & (mgb['loca']=='강남구'),'pm25_mean'])
    
    

In [ ]:
plt.subplots(figsize=(25,8))
b = sns.boxplot(data = ds[(ds['loca']=='강남구')],
                hue = 'y',
                x = 'm',
                y = 'pm10'                )

In [ ]:
med_dict = {}
min_dict = {}
max_dict = {}



In [ ]:
print(type(ds.dt[0]))
plt.subplots(figsize=(16,5))
# plt.xticks(ds.loc[(ds['dt'].dt.year==2015) & (ds['loca']=='강남구'),'dt'].tolist(), rotation=90)
for yee in y2u:
    plt.plot(ds.loc[(ds['dt'].dt.year==yee) & (ds['loca']=='강동구'),'day'],\
             ds.loc[(ds['dt'].dt.year==yee) & (ds['loca']=='강동구'),'pm25'])

In [ ]:
print(ds.loc[(ds['dt'].dt.year==2017) & (ds['loca']=='강동구'),'dt'].shape)
print(ds.loc[(ds['dt'].dt.year==2018) & (ds['loca']=='강동구'),'dt'].shape)

In [ ]:
ds.loc[(ds['dt'].dt.year==2015) & (ds['loca']=='강남구'),:].sort_values('pm10',ascending=False).head()

In [ ]:
# Your data, with some additional elements deleted...
data = np.array(
[[  5., 22.57011604],
 [  6., 22.57408142],
 [  9., 22.56790352],
 [ 10., 22.56451225],
 [ 11., 22.56481743],
 [ 12., 22.55789757],
 [ 29., 22.55654716],
 [ 33., 22.56213379],
 [ 34., 22.5588932 ],
 [ 35., 22.54829407],
 [ 40., 22.55079269],
 [ 41., 22.55475616],
 [ 41., 22.54783821],
 [ 42., 22.55195618]])

x = data[:,0]
y = data[:,1]

# Difference from element to element in x
dx = x[1:]-x[:-1]

In [ ]:
x[1:]

In [ ]:
x[:-1]

In [ ]:
a = {}
a['layout'] = {}
a['layout'].update(title='kkk')

In [ ]:
for i,v in enumerate(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']):
    print(i,v)

In [ ]:
a = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
a[2:10]

In [ ]:
for i in range(9):
    print(i)